In [ ]:
%clear all
import numpy as np
import matplotlib.pyplot as plt

In [95]:
!rm -rf ./data
!mkdir -p ./data/param ./data/traj

First things first, let's compile the program

In [ ]:
!gcc ./src/* -o oscill -lm -Wall -I ./include/

In [85]:
def generate_input_file(params):
    content = f"""k {params.get("k", 0)}
G {params.get("G", 0)}
F {params.get("F", 0)}
W {params.get("W", 0)}
x0 {params.get("x0", 0)}
v0 {params.get("v0", 0)}
dt {params.get("dt", 0.01)}
T {params.get("T", 1)}"""
    return content

def save_input_file(content, filename):
    with open(filename, "w") as file:
        file.write(content)

**Parameters for the ideal harmonic oscillator**

In [86]:
params_IHO = {
    "k": 25,
    "G": 0,
    "F": 0,
    "W": 0,
    "x0": 1.0,
    "v0": 0.0,
    "dt": 0.001,
    "T": 20
}
content = generate_input_file(params_IHO)
filename = "data/param/idealHO.params"
save_input_file(content, filename)

!./oscill $filename > "./data/traj/data_G0_F0.dat"

**Check that the energy is conserved**

In [ ]:
data_ideal = np.loadtxt("data/traj/data_G0_F0.dat", delimiter=",")

fig, ax = plt.subplots(1, 1, figsize=(1.618*5, 5))

ax.plot(data_ideal[:, 0], data_ideal[:, 3]-data_ideal[:, 3].mean())
ax.set_ylabel("E(t)-<E(t)>", fontsize=15)
ax.set_xlabel("Time", fontsize=15)
ax.grid()

**Parameters for the dumped harmonic oscillator**

In [88]:
params_IHO = {
    "k": 25,
    "G": 0.5,
    "F": 0,
    "W": 0,
    "x0": 1.0,
    "v0": 0.0,
    "dt": 0.001,
    "T": 20
}
content = generate_input_file(params_IHO)
filename = "data/param/dumpedHO.params"
save_input_file(content, filename)

!./oscill $filename > "./data/traj/data_G0.5_F0.dat"

**Energy profile for dumped**

In [ ]:
data_dumped = np.loadtxt("data/traj/data_G0.5_F0.dat", delimiter=",")

fig, ax = plt.subplots(1, 1, figsize=(1.618*5, 5))

ax.plot(data_dumped[:, 0], data_dumped[:, 3])
ax.set_ylabel("E(t)", fontsize=15)
ax.set_xlabel("Time", fontsize=15)
ax.grid()

**Parameters for the dumped harmonic oscillator with driving force**

In [ ]:
params = {
    "k": 25,
    "G": 0.5,
    "F": 10,
    "W": 0,
    "x0": 1.0,
    "v0": 0.0,
    "dt": 0.001,
    "T": 20
}

for omega in np.arange(4, 6+0.1, 0.1):
    print(f"Running for omega = {omega:.2f}", end="\r")
    params["W"] = omega
    content = generate_input_file(params)
    
    filename = f"data/param/drivenHO_W{omega:.2f}.params"
    save_input_file(content, filename)

    output_dat = f"data/traj/data_G0.5_F1_W{omega:.2f}.dat"

    !./oscill $filename > $output_dat

**Rapporto tra Ampiezze di Oscillazioni**

Utilizziamo il file `data/data_G0_F0.dat` come riferimento. In alternativa si potrebbe utilizzare la soluzione analitica.

In [91]:
def amplitude_ratio(t, Tmin, ideal, dumped_driven):
    mask = t >= Tmin
    ideal = ideal[mask]
    dumped_driven = dumped_driven[mask]
    return t[mask], dumped_driven / ideal

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(1.618*5, 5))

for omega in np.arange(4, 6+0.1, 0.1):
    input_dat = f"data/traj/data_G0.5_F1_W{omega:.2f}.dat"

    data_dumped_driven = np.loadtxt(input_dat, delimiter=",")
    t, ar = amplitude_ratio(data_ideal[:, 0], 15, data_ideal[:, 3], data_dumped_driven[:, 3])
    
    ax.plot(t, ar, label=fr"$\omega={omega:.2f}$")

ax.set_ylabel(r"$\frac{A(t)}{A_{id}(t)}$", fontsize=15, rotation=0, labelpad=20)
ax.set_xlabel("Time", fontsize=15)
ax.legend()
ax.grid()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(1.618*5, 5))

ax.plot(data_ideal[:, 0], data_ideal[:, -1], label=r"$E_{HO}$")

for omega in np.arange(4, 6+0.1, 0.1):
    input_dat = f"data/traj/data_G0.5_F1_W{omega:.2f}.dat"

    data_dumped_driven = np.loadtxt(input_dat, delimiter=",")
    ax.plot(data_dumped_driven[:, 0], data_dumped_driven[:, -1], label=fr"$E_{{\omega={omega:.2f}}}$")

ax.set_ylabel(r"$E(t)$", fontsize=15, rotation=0, labelpad=20)
ax.set_xlabel("Time", fontsize=15)
ax.legend()
ax.grid()

In [ ]:
# https://phys.libretexts.org/Bookshelves/University_Physics/Physics_(Boundless)/15%3A_Waves_and_Vibrations/15.4%3A_Damped_and_Driven_Oscillations
omega0 = np.sqrt(params["k"]/1)
print(f"Omega_0 = {omega0:.2f}")

beta = params["G"]/(2*1.0)
print(f"Beta = {beta:.2f}")

omega_r = np.sqrt(omega0**2 - 2*beta**2)
print(f"Omega_r = {omega_r:.3f}")